# Migrating from `uncon_conjgrd_comp` to `handle_solve_bounds_foas`

This notebook illustrates the steps required to upgrade from the solver `uncon_conjgrd_comp` (`E04DG`) to `handle_solve_bounds_foas` (`E04KF`) introduced at Mark 27 of the NAG Library.

From the usage perspective, the main difference between the solvers is the user call-backs,
`uncon_conjgrd_comp` has a single user call-back that can return *objective 
function* and *gradient* evaluations, while `handle_solve_bounds_foas` has two separate user call-backs, 
one for the *objective funtion* and one for the *objective gradient*.

In this notebook the 2d Rosenbrock problem is solved with both solvers and illustrates the changes necessary for the migration to `handle_solve_bounds_foas`. The solution to the problem is (1, 1).

In [1]:
# NAG Copyright 2020.
from naginterfaces.base import utils
from naginterfaces.library import opt
import numpy as np

In [2]:
# Define E04DG user call-back
def objfun_e04dg(mode, x, _nstate, _data=None):
    objf = (1. - x[0])**2 + 100.*(x[1] - x[0]**2)**2
    if mode == 2:
        fdx = [
            2.*x[0] - 400.*x[0]*(x[1]-x[0]**2) - 2.,
            200.*(x[1]-x[0]**2),
        ]
        return objf, fdx
    return objf, np.zeros(len(x))

In [3]:
# Define user call-backs for E04KF
def objfun_e04kf(x, inform, _data=None): 
    """Return the objective function value"""
    objf = (1. - x[0])**2 + 100.*(x[1] - x[0]**2)**2
    return objf, inform

def objgrd_e04kf(x, fdx, inform, _data=None):
    """The objective's gradient. Note that fdx has to be updated IN-PLACE"""
    fdx[:] = [
        2.*x[0] - 400.*x[0]*(x[1]-x[0]**2) - 2.,
        200.*(x[1]-x[0]**2),
    ]
    return inform

In [4]:
# The initial guess
x = [-1.5, 1.9]

# Use an explicit I/O manager for abbreviated iteration output
iom = utils.FileObjManager(locus_in_output=False)

## Solve the problem with `uncon_conjgrd_comp`

In [5]:
comm = opt.nlp1_init('uncon_conjgrd_comp')
slv = opt.uncon_conjgrd_comp(objfun_e04dg, x, comm, io_manager=iom)
print('Solution: \n', slv.x)

<ipython-input-5-a8bdc418615c>:2: NagDeprecatedWarning: (NAG Python function naginterfaces.library.opt.uncon_conjgrd_comp)
This function is deprecated.
The following advice is given for making a replacement:
Please use handle_solve_bounds_foas instead.
See also https://www.nag.com/numeric/py/nagdoc_latest/replace.html
  slv = opt.uncon_conjgrd_comp(objfun_e04dg, x, comm, io_manager=iom)


Solution: 
 [1.00000676 1.00001354]


## Now solve with the new solver `handle_solve_bounds_foas`

In [6]:
# Create an empty handle for the problem
nvar = len(x)
handle = opt.handle_init(nvar)

# Define the nonlinear objective in the handle
# Setup a gradient vector of length nvar
opt.handle_set_nlnobj(handle, idxfd=list(range(1, nvar+1)))

# Set some algorithmic options
for option in [
        'Print Options = No',      # print Options?
        'Print Solution = Yes',    # print on the screen the solution point X
        'Print Level = 1',         # print details of each iteration (screen)
]:
    opt.handle_opt_set(handle, option)
    
# Solve the problem and print the solution
opt.handle_solve_bounds_foas(handle, x, objfun=objfun_e04kf, objgrd=objgrd_e04kf,
        io_manager=iom)

# Destroy the handle and free allocated memory
opt.handle_free(handle)

 E04KF, First order method for bound-constrained problems

 Status: converged, an optimal solution was found
 Value of the objective             2.12807E-15
 Norm of gradient                   3.67342E-08

 Primal variables:
   idx   Lower bound       Value       Upper bound
     1       -inf        1.00000E+00        inf
     2       -inf        1.00000E+00        inf
